<a href="https://colab.research.google.com/github/Gokul-06/Staples/blob/main/Staples_Customer_Update.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
  pip install yfinance psycopg2


In [2]:
import yfinance as yf

# Fetch data
def fetch_stock_data(symbol):
    stock = yf.Ticker(symbol)
    hist = stock.history(period="1d")
    last_quote = hist.iloc[-1] # Get the most recent data point
    return {
        'symbol': symbol,
        'price': last_quote['Close'],
        'volume': last_quote['Volume']
    }

# Example usage
stock_data = fetch_stock_data('PG')
print(stock_data)


{'symbol': 'PG', 'price': 155.27000427246094, 'volume': 2096311.0}


In [5]:
import mysql.connector
import yfinance as yf

def fetch_stock_data(symbol):
    stock = yf.Ticker(symbol)
    hist = stock.history(period="1d")
    last_quote = hist.iloc[-1]
    return {
        'symbol': symbol,
        'price': last_quote['Close'],
        'volume': last_quote['Volume']
    }

def update_database(stock_data):
    # Connect to your SingleStore database with the provided details
    connection = mysql.connector.connect(
        user='admin',
        password='Rules123',
        host='svc-538d9a8e-2af0-4a3c-abf5-f797b0d6b9f0-dml.aws-virginia-6.svc.singlestore.com',
        database='Staples',  # Replace 'your_database_name' with the name of your database
        port=3306
    )
    cursor = connection.cursor()

    # Insert or update stock data
    update_query = """INSERT INTO stock_prices (symbol, price, volume) VALUES (%s, %s, %s)
                      ON DUPLICATE KEY UPDATE price = VALUES(price), volume = VALUES(volume);"""
    cursor.execute(update_query, (stock_data['symbol'], stock_data['price'], stock_data['volume']))

    connection.commit()
    cursor.close()
    connection.close()

# Example usage
stock_data = fetch_stock_data('PG')
update_database(stock_data)


In [3]:
pip install mysql-connector-python yfinance

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.5/21.5 MB 38.3 MB/s eta 0:00:00


In [6]:
import yfinance as yf
import mysql.connector
from datetime import datetime

def fetch_stocks_data(symbols):
    stock_data = []
    # yfinance can handle multiple tickers at once, but consider splitting
    # the request if it's too large
    for i in range(0, len(symbols), 100):  # Example: Split list into chunks of 100
        chunk = symbols[i:i + 100]
        stocks = yf.Tickers(" ".join(chunk))
        hist = stocks.history(period="1d")

        for symbol in chunk:
            data = hist.get(symbol)
            if data is not None and not data.empty:
                last_quote = data.iloc[-1]
                stock_data.append({
                    'symbol': symbol,
                    'price': last_quote['Close'],
                    'volume': last_quote['Volume'],
                    'last_update': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
                })
    return stock_data

def update_database_bulk(stock_data):
    connection = mysql.connector.connect(
        user='admin', password='Rules123',
        host='svc-538d9a8e-2af0-4a3c-abf5-f797b0d6b9f0-dml.aws-virginia-6.svc.singlestore.com',
        database='stoc', port=3306)
    cursor = connection.cursor()

    update_query = """
    INSERT INTO stock_prices (symbol, price, volume, last_update)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE price = VALUES(price), volume = VALUES(volume), last_update = VALUES(last_update);
    """

    # Bulk update
    cursor.executemany(update_query, [(d['symbol'], d['price'], d['volume'], d['last_update']) for d in stock_data])

    connection.commit()
    cursor.close()
    connection.close()

# Assuming you have a list of S&P 500 stock symbols
symbols = ['WMT', 'PG', 'COST']  # This should be your actual list of S&P 500 symbols
stock_data = fetch_stocks_data(symbols)
update_database_bulk(stock_data)

[*********************100%%**********************]  3 of 3 completed


In [7]:
symbols = ['WMT', 'PG', 'COST', 'NSRGY', 'KO', 'LRLCY', 'PEP', 'PM', 'UL', 'BUD']


In [8]:
!pip install yfinance mysql-connector-python

import time
from datetime import datetime
import yfinance as yf
import mysql.connector

# Corrected function to fetch stock data
def fetch_stocks_data(symbols):
    stock_data = []
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="1d")
        if not hist.empty:
            last_quote = hist.iloc[-1]
            stock_data.append({
                'symbol': symbol,
                'price': last_quote['Close'],
                'volume': last_quote['Volume'],
                'last_update': datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            })
        else:
            print(f"No data found for {symbol}, symbol may be delisted")
    return stock_data

# Function to update the database in bulk
def update_database_bulk(stock_data):
    connection = mysql.connector.connect(
        user='admin', password='Rules123',
        host='svc-538d9a8e-2af0-4a3c-abf5-f797b0d6b9f0-dml.aws-virginia-6.svc.singlestore.com',
        database='Staples')  # Replace 'your_database_name' with your actual database name
    cursor = connection.cursor()
    update_query = """
    INSERT INTO stock_prices (symbol, price, volume, last_update)
    VALUES (%s, %s, %s, %s)
    ON DUPLICATE KEY UPDATE price = VALUES(price), volume = VALUES(volume), last_update = VALUES(last_update);
    """
    cursor.executemany(update_query, [(d['symbol'], d['price'], d['volume'], d['last_update']) for d in stock_data])
    connection.commit()
    cursor.close()
    connection.close()

if __name__ == "__main__":
    symbols = ['WMT', 'PG', 'COST', 'NSRGY', 'KO', 'LRLCY', 'PEP', 'PM', 'UL', 'BUD']
    try:
        while True:
            stock_data = fetch_stocks_data(symbols)
            if stock_data:  # Check if there's data to update
                update_database_bulk(stock_data)
                print(f"Updated at {datetime.now()}")
            else:
                print("No data fetched for update.")
            time.sleep(10)  # Sleep for 60 seconds before the next update
    except KeyboardInterrupt:
        print("Manual interrupt, stopping updates.")


Updated at 2024-04-12 16:53:51.547444
Updated at 2024-04-12 16:54:02.991635
Updated at 2024-04-12 16:54:14.357829
Manual interrupt, stopping updates.
